In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI TensorBoard 集成 Vertex AI 管道

<table align="left">

  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tensorboard/tensorboard_vertex_ai_pipelines_integration.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> <br> 在 Colab 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Ftensorboard%2Ftensorboard_vertex_ai_pipelines_integration.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"> <br> 在 Colab Enterprise 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tensorboard/tensorboard_vertex_ai_pipelines_integration.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"> <br>
      在 GitHub 上查看
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tensorboard/tensorboard_vertex_ai_pipelines_integration.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> <br>
      在 Vertex AI Workbench 中打开
    </a>
  </td>                                                                                               
</table>

## 概述

### 什么是 Vertex AI TensorBoard

Vertex AI TensorBoard 是[开源TensorBoard](https://www.tensorflow.org/tensorboard/get_started)（TB）的企业级托管版本，TensorBoard是谷歌为机器学习实验可视化开发的开源项目。

Vertex AI TensorBoard 提供各种详细的可视化，包括：

*   跟踪和可视化指标，如随时间变化的损失和准确率。
*   可视化模型计算图（操作和层）。
*   查看随时间变化的权重、偏置或其他张量的直方图。
*   投影嵌入到低维空间。
*   显示图像、文本和音频样本。

除了TensorBoard提供的强大可视化之外，Vertex AI TensorBoard 还提供以下优势：

*  持久的、可共享的实验仪表板链接。

*  项目中所有实验的可搜索列表。

*  与 Vertex AI 服务的模型训练紧密集成。

*  企业级安全、隐私和合规性。

通过 Vertex AI TensorBoard，您可以跟踪、可视化和比较机器学习实验，并与团队共享。

了解更多关于[Vertex AI TensorBoard](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-introduction)和[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)。

### 目标

在本教程中，您将学习如何使用KFP SDK创建训练管道，在Vertex AI Pipelines中执行管道，并在Vertex AI TensorBoard上实时监控培训过程。

本教程使用以下谷歌云ML服务和资源：

- Vertex AI训练
- Vertex AI TensorBoard
- Vertex AI Pipelines

执行的步骤包括：

* 设置服务帐户和谷歌云存储桶。
* 使用您的自定义训练代码构建一个KFP管道。
* 编译并在启用Tensorboard的情况下在Vertex AI Pipelines中执行KFP管道，以进行几乎实时监控。

数据集

本教程使用的数据集将是由 TensorFlow 提供的[花卉数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。不需要其他数据集。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

## 安装

安装以下所需的软件包以执行此笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 "kfp<2" \
                                 "google-cloud-pipeline-components==1.0.20"

### 仅适用于Colab：请取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的谷歌云项目

**无论您使用任何笔记本环境，都需要执行以下步骤。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得300美元的免费信用额度用于计算/存储成本。

2. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您是在本地运行这个笔记本，请安装 [Cloud SDK](https://cloud.google.com/sdk)。

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

设置区域

**可选**：更新'REGION'变量以指定您要使用的区域。了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### UUID

UUID 是通用唯一标识符（Universally Unique Identifier）的简称，是一种用于识别信息的标准化方法。

如果您正在进行实时教程，您可能正在使用共享的测试账户或项目。
为了避免用户之间资源命名相冲突，为每个实例会话创建一个通用唯一标识符（UUID）。
在本教程中创建的资源名称后附加UUID。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google云账户

要验证您的Google云账户，请按照您的Jupyter环境的指示进行操作。

**Vertex AI 工作台**
<br>您已经通过身份验证。

本地JupyterLab实例
取消注释并运行以下代码：

In [ ]:
# ! gcloud auth login

* **Colab**
<br>取消注释并运行以下代码：

In [ ]:
# from google.colab import auth

# auth.authenticate_user()

请查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上为您的服务帐户授予云存储权限。

### 创建一个云存储桶

创建一个存储桶来存储中间产物，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在： 运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

设置服务账号和权限

一个服务账户用来创建自定义培训任务。如果你不想使用项目的计算引擎服务账户，请将SERVICE_ACCOUNT设置为另一个服务账户ID。你可以按照文档指示创建一个服务账户。 (https://cloud.google.com/iam/docs/creating-managing-service-accounts#creating)。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = ! gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

In [ ]:
# Grant Cloud Storage permission.
! gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT \
    --role=roles/storage.admin \
    --quiet

In [ ]:
# Grant AI Platform permission.
! gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT \
    --role=roles/aiplatform.user \
    --quiet

### 导入ai平台

In [ ]:
import google.cloud.aiplatform as aiplatform

初始化用于Python的Vertex AI SDK
为您的项目和相应的存储桶初始化用于Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

将以下常量设置为 Vertex AI Pipelines 常量：

In [ ]:
PIPELINE_ROOT = "{}/tensorboard-pipeline-integration/pipeline_root/".format(BUCKET_URI)
BASE_OUTPUT_DIR = "{}/pipeline-output/tensorboard-pipeline-integration-{}".format(
    BUCKET_URI, UUID
)

额外的进口。

In [ ]:
from google_cloud_pipeline_components.v1.custom_job.utils import \
    create_custom_training_job_op_from_component
from kfp.v2 import dsl
from kfp.v2.dsl import component

创建一个Vertex AI Tensorboard实例

创建一个供Pipeline使用的TensorBoard实例。

In [ ]:
TENSORBOARD_NAME = "[your-tensorboard-name]"  # @param {type:"string"}

if (
    TENSORBOARD_NAME == ""
    or TENSORBOARD_NAME is None
    or TENSORBOARD_NAME == "[your-tensorboard-name]"
):
    TENSORBOARD_NAME = PROJECT_ID + "-tb-" + UUID

tensorboard = aiplatform.Tensorboard.create(
    display_name=TENSORBOARD_NAME, project=PROJECT_ID, location=REGION
)
TENSORBOARD_RESOURCE_NAME = tensorboard.gca_resource.name
print("TensorBoard resource name:", TENSORBOARD_RESOURCE_NAME)

## 定义基于函数的 Python 管道训练器组件
在本教程中，您将定义基于函数的组件来训练模型。
训练代码被包装为一个在 Vertex AI Pipeline 中运行的 KFP 组件。

您的训练代码必须配置为将 TensorBoard 日志写入 Cloud Storage 存储桶，
Vertex AI 训练服务会自动使用预定义的环境变量 `AIP_TENSORBOARD_LOG_DIR` 提供存储位置。

通常可以通过提供 `os.environ['AIP_TENSORBOARD_LOG_DIR']` 作为用于写入开源 TensorBoard 日志的日志目录来实现此目标。

例如，在 TensorFlow 2.x 中，您可以使用以下代码创建一个 `tensorboard_callback`：
```
tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'],
  histogram_freq=1)
```
并将回调函数添加到 model.fit(...) 中
```
# 前面的代码
model.compile(...)

tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'],
  histogram_freq=1)
  
model.fit(dataset, epochs=10, callbacks=[tensorboard_callback])
```

In [ ]:
@component(
    base_image="tensorflow/tensorflow:latest",
    packages_to_install=["tensorflow_datasets"],
)
def trainer(tb_log_dir_env_var: str = "AIP_TENSORBOARD_LOG_DIR"):
    """Training component."""
    import logging
    import os

    import tensorflow as tf
    import tensorflow_datasets as tfds

    IMG_WIDTH = 128

    def normalize_img(image):
        """Normalizes image.

        * Resizes image to IMG_WIDTH x IMG_WIDTH pixels
        * Casts values from `uint8` to `float32`
        * Scales values from [0, 255] to [0, 1]

        Returns:
          A tensor with shape (IMG_WIDTH, IMG_WIDTH, 3). (3 color channels)
        """
        image = tf.image.resize_with_pad(image, IMG_WIDTH, IMG_WIDTH)
        return image / 255.0

    def normalize_img_and_label(image, label):
        """Normalizes image and label.

        * Performs normalize_img on image
        * Passes through label unchanged

        Returns:
          Tuple (image, label) where
          * image is a tensor with shape (IMG_WIDTH, IMG_WIDTH, 3). (3 color
            channels)
          * label is an unchanged integer [0, 4] representing flower type
        """
        return normalize_img(image), label

    if "AIP_MODEL_DIR" not in os.environ:
        raise KeyError(
            "The `AIP_MODEL_DIR` environment variable has not been"
            + "set. See https://cloud.google.com/ai-platform-unified/docs/tutorials/image-recognition-custom/training"
        )
    output_directory = os.environ["AIP_MODEL_DIR"]

    logging.info("Loading and preprocessing data ...")
    dataset = tfds.load(
        "tf_flowers:3.*.*",
        split="train",
        try_gcs=True,
        shuffle_files=True,
        as_supervised=True,
    )
    dataset = dataset.map(
        normalize_img_and_label, num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    dataset = dataset.cache()
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(128)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    logging.info("Creating and training model ...")
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Conv2D(
                16,
                3,
                padding="same",
                activation="relu",
                input_shape=(IMG_WIDTH, IMG_WIDTH, 3),
            ),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation="relu"),
            tf.keras.layers.Dense(5),  # 5 classes
        ]
    )
    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    # Create a TensorBoard call back and write to the gcs path provided by AIP_TENSORBOARD_LOG_DIR
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=os.environ[tb_log_dir_env_var], histogram_freq=1
    )

    # Train the model with tensorboard_callback
    model.fit(dataset, epochs=14, callbacks=[tensorboard_callback])

    logging.info(f"Exporting SavedModel to: {output_directory}")
    # Add softmax layer for intepretability
    probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
    probability_model.save(output_directory)

定义一个使用您的组件的管道

接下来，定义一个使用在上一节中构建的组件的管道。

`create_custom_training_job_op_from_component`函数将给定的组件转换为 Vertex AI 中的定制训练作业（`CustomTrainingJobOp`）。

In [ ]:
@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline. Use to determine the pipeline Context.
    name="tb-pipeline-integration",
)
def pipeline():
    custom_job_op = create_custom_training_job_op_from_component(
        trainer,
        tensorboard=TENSORBOARD_RESOURCE_NAME,
        base_output_directory=BASE_OUTPUT_DIR,
        service_account=SERVICE_ACCOUNT,
    )
    custom_job_op(project=PROJECT_ID, location=REGION)

编译管道。

In [ ]:
from kfp.v2 import compiler  # noqa: F811

compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="tensorboard-pipeline-integration.json"
)

运行管道

接下来，运行管道。

In [ ]:
DISPLAY_NAME = "tb-pipeline-integration_" + UUID

job = aiplatform.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="tensorboard-pipeline-integration.json",
    pipeline_root=PIPELINE_ROOT,
)

job.run()

! rm tensorboard-pipeline-integration.json

检查训练日志

Vertex AI TensorBoard网络应用程序提供了与Vertex AI TensorBoard实验相关的日志的可视化。该网络应用程序提供了多个工具和仪表板，以可视化和比较实验运行中的数据。

了解更多，请参阅[查看Vertex AI TensorBoard数据](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-view)。

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，**如果您在笔记本中创建了单独的资源**，您可以按以下方式删除这些资源：

In [ ]:
# Delete GCS bucket.
! gsutil -m rm -r {BUCKET_URI}

# Delete TensorBoard instance.
! gcloud ai tensorboards delete {TENSORBOARD_RESOURCE_NAME}

# Delete custom job.
job.delete()